In [89]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from helper_functions import accuracy_fn
from safetensors.torch import load_model
from PIL import Image
import matplotlib.pyplot as plt

In [90]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### Data Processing

In [91]:
train_df = pd.read_csv("data/EMNIST/raw/emnist-balanced-train.csv")
test_df =  pd.read_csv("data/EMNIST/raw/emnist-balanced-test.csv")

In [92]:
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.Grayscale(3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [94]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, feature_columns, target_column=None, transform=None):
        self.dataframe = dataframe
        self.feature_columns = feature_columns
        self.target_column = target_column
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        reshaped_img = train_df.iloc[idx][self.feature_columns].values.reshape((28, 28)).astype(np.uint8)
        pil_img = Image.fromarray(reshaped_img)
        features = data_transforms(pil_img)
        
        # Get target if specified
        if self.target_column is not None:
            target = torch.tensor(self.dataframe.iloc[idx][self.target_column]).type(torch.LongTensor).to(device)
            return features, target
        
        # Return only features if no target column is provided
        return features

In [95]:
train_data = CustomDataset(train_df, train_df.columns[1:], train_df.columns[0])
test_data = CustomDataset(test_df, test_df.columns[1:], test_df.columns[0])

In [96]:
batch_size = 16

# put custom dataset to dataloader
train_dl = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [97]:
# get how many classes are there
classes = pd.unique(train_df["45"].values)
len(classes)

47

In [102]:
# check shapes of dataloader
feature, label = next(iter(test_dl))
feature.shape, label.shape

KeyError: "['76', '151', '19.1', '156', '253', '46', '129', '18.1', '38', '250.1', '126', '11', '141', '250.2', '126.1', '214', '250.3', '250.4', '253.1', '6', '37.4', '35', '253.2', '189', '145', '129.1', '191', '250.5', '189.1', '251.2', '90', '218', '251.3', '242.1', '132', '64', '41.1', '146', '152', '13', '203.2', '235', '250.6', '32.3', '253.3', '191.1', '116', '32.4', '140', '159.1', '16', '46.1', '115.1', '177.1', '221', '253.4', '251.4', '32.5', '251.5', '148', '38.1', '37.5', '64.1', '209', '250.7', '147', '44', '201', '79', '5.2', '207.2', '251.6', '222.1', '236.1', '218.1', '253.5', '250.8', '164', '32.6', '7.5', '7.6', '78', '176', '251.7', '115.2', '47', '247.4', '247.5', '222.2', '170.1', '9', '5.3', '47.1', '249.3', '245.2', '220.1', '159.2', '52.1', '37.6', '21.2', '10.1', '91.1', '202.2', '204.1', '139.1', '11.1', '7.7', '32.7', '37.7', '32.8', '9.1'] not in index"

### Load Model

In [99]:
model = torch.hub.load('pytorch/vision:v0.10.0', "resnet18", pretrained=False)
model.to(device)
load_model(model, "resnet_ocr.safetensors")

Using cache found in C:\Users\paoma/.cache\torch\hub\pytorch_vision_v0.10.0
c:\Users\paoma\miniconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\paoma\miniconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


(set(), [])

In [100]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.01)

### Train Model

In [101]:
torch.manual_seed(20)

epochs = 5

for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n------")
    
    # TRAINING
    train_loss, train_acc = 0, 0
    model.train()
    for batch, (X, y) in enumerate(train_dl):

        X, y = X.to(device), y.to(device)

        # forward pass
        train_pred = model(X)

        # metrics
        loss = loss_fn(train_pred, y)
        train_loss += loss
        train_acc += accuracy_fn(y_true=y, y_pred=train_pred.argmax(dim=1))

        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # print metrics
    train_loss /= len(train_dl)
    train_acc /= len(train_dl)
    print(f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_acc:.2f}%")

    
    # TESTING
    test_loss, test_acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for X, y in test_dl:

            X, y = X.to(device), y.to(device)


            # forward pass
            test_pred = model(X)

            # metrics
            test_loss += loss_fn(test_pred, y)
            test_acc += accuracy_fn(y_true=y, y_pred=test_pred.argmax(dim=1))
        
        # print metrics
        test_loss /= len(test_dl)
        test_acc /= len(test_dl)
        print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.2f}%")

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0
------


  0%|          | 0/5 [08:34<?, ?it/s]

Train Loss: 0.4403 | Train Accuracy: 85.76%


KeyError: "['76', '151', '19.1', '156', '253', '46', '129', '18.1', '38', '250.1', '126', '11', '141', '250.2', '126.1', '214', '250.3', '250.4', '253.1', '6', '37.4', '35', '253.2', '189', '145', '129.1', '191', '250.5', '189.1', '251.2', '90', '218', '251.3', '242.1', '132', '64', '41.1', '146', '152', '13', '203.2', '235', '250.6', '32.3', '253.3', '191.1', '116', '32.4', '140', '159.1', '16', '46.1', '115.1', '177.1', '221', '253.4', '251.4', '32.5', '251.5', '148', '38.1', '37.5', '64.1', '209', '250.7', '147', '44', '201', '79', '5.2', '207.2', '251.6', '222.1', '236.1', '218.1', '253.5', '250.8', '164', '32.6', '7.5', '7.6', '78', '176', '251.7', '115.2', '47', '247.4', '247.5', '222.2', '170.1', '9', '5.3', '47.1', '249.3', '245.2', '220.1', '159.2', '52.1', '37.6', '21.2', '10.1', '91.1', '202.2', '204.1', '139.1', '11.1', '7.7', '32.7', '37.7', '32.8', '9.1'] not in index"